In [1]:
import pandas as pd, os, numpy as np

In [5]:
EU_countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands (Kingdom of the), Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden".split(", ")
os.chdir(r"C:\Users\Melanie\OneDrive - Rensselaer Polytechnic Institute\Desktop\School\24_Spring\Data Analytics\DA_Project\Old Datasets")

In [3]:
def fill_blanks(df):
    """This takes a dataframe (assuming there's a column called "Country") and fills in all the blank country and technologyspots"""
    all_countries = list(df["Country"].dropna())

    for i in range(len(all_countries)-1):
        start_index = df[df["Country"] == all_countries[i]].index[0]
        end_index = df[df["Country"] == all_countries[i+1]].index[0]
        df.loc[start_index:end_index-1, "Country"] = all_countries[i]
        
    #fill the last subset
    df["Country"] = df["Country"].fillna(all_countries[-1])

    #get just the EU countries
    df = df[df["Country"].isin(EU_countries)]

    #time to fill in the technologies
    n = len(EU_countries)
    technologies = list(df[df["Country"] == EU_countries[0]]["Technology"].dropna())
    new_df = df.copy(deep=True)

    for j in range(n): #for each country
        for i in range(len(technologies)-1): #for each technology
            #find the location where the 
            start_index = df[df["Technology"] == technologies[i]].index[j]
            end_index = df[df["Technology"] == technologies[i+1]].index[j]

            new_df.loc[start_index:end_index-1, "Technology"] = technologies[i]
        
    new_df["Technology"].fillna(technologies[-1],inplace=True)
    return new_df

#### Doing the ongrid electricity

In [6]:
amount_str = "On-grid Capacity (MW)"
ongrid = pd.read_excel("InstalledElectricityCapacity_Ongrid.xlsx",skiprows=1)

#remove the last few rows
ongrid.drop(ongrid.tail(26).index, inplace=True)

#get rid of the column that just says "on-grid"
ongrid.drop("Unnamed: 2",inplace=True,axis=1)

#rename the columns
ongrid = ongrid.rename(columns = {"Unnamed: 0": "Country",
                                  "Unnamed: 1": "Technology",
                                  "Unnamed: 3": "Year",
                                  "Unnamed: 4": amount_str})

In [7]:
new_ongrid = fill_blanks(ongrid)

In [8]:
new_new_ongrid = new_ongrid.pivot_table(index=["Country","Year"], columns='Technology', values=amount_str,aggfunc='sum')

In [9]:
new_new_ongrid

Technology      Biogas  Coal and peat  Fossil fuels n.e.s.  Geothermal energy  \
Country Year                                                                    
Austria 2000.0    24.0            0.0               5247.0                0.0   
        2001.0    27.0            0.0               5407.0                0.0   
        2002.0    18.0            0.0               5257.0                2.0   
        2003.0    29.0            0.0               4922.0                2.0   
        2004.0    29.0            0.0               5138.0                2.0   
...                ...            ...                  ...                ...   
Sweden  2018.0     2.0            0.0               2632.0                0.0   
        2019.0     2.0            0.0               2287.0                0.0   
        2020.0   100.0            0.0               4119.0                0.0   
        2021.0   100.0            0.0               3760.0                0.0   
        2022.0   100.0            0.0               3760.0                0.0   

Technology      Liquid biofuels  Marine energy  Mixed Hydro Plants  \
Country Year                                                         
Austria 2000.0              0.0           0.00              3937.0   
        2001.0              0.0           0.00              3937.0   
        2002.0              2.0           0.00              3937.0   
        2003.0             10.0           0.00              3938.0   
        2004.0             17.0           0.00              3965.0   
...                         ...            ...                 ...   
Sweden  2018.0            535.0           0.19                99.0   
        2019.0            895.0           0.00                99.0   
        2020.0            895.0           0.00                99.0   
        2021.0            895.0           0.00                99.0   
        2022.0            895.0           0.00                99.0   

Technology      Natural gas  Nuclear  Offshore wind energy  Oil  \
Country Year                                                      
Austria 2000.0          0.0      0.0                   0.0  0.0   
        2001.0          0.0      0.0                   0.0  0.0   
        2002.0          0.0      0.0                   0.0  0.0   
        2003.0          0.0      0.0                   0.0  0.0   
        2004.0          0.0      0.0                   0.0  0.0   
...                     ...      ...                   ...  ...   
Sweden  2018.0          0.0   8656.0                 203.0  0.0   
        2019.0          0.0   8624.0                 203.0  0.0   
        2020.0          0.0   7777.0                 203.0  0.0   
        2021.0          0.0   6899.0                 193.0  0.0   
        2022.0          0.0   6899.0                 193.0  0.0   

Technology      Onshore wind energy  Other non-renewable energy  \
Country Year                                                      
Austria 2000.0                 50.0                        77.0   
        2001.0                 67.0                       189.0   
        2002.0                109.0                       193.0   
        2003.0                322.0                       374.0   
        2004.0                581.0                       404.0   
...                             ...                         ...   
Sweden  2018.0               7097.0                       755.5   
        2019.0               8478.0                       757.5   
        2020.0               9773.0                       622.5   
        2021.0              11923.0                       386.0   
        2022.0              14364.0                       386.0   

Technology      Pumped storage  Renewable hydropower  \
Country Year                                           
Austria 2000.0             0.0                7676.0   
        2001.0             0.0                7703.0   
        2002.0             0.0                7567.0   
        200

In [137]:
new_new_ongrid.to_csv("ongrid_cap.csv")

In [138]:
(new_new_ongrid == 0).astype(int).sum(axis=0)/np.shape(new_new_ongrid)[0]

Technology
Biogas                        0.078905
Coal and peat                 1.000000
Fossil fuels n.e.s.           0.027375
Geothermal energy             0.809984
Liquid biofuels               0.771337
Marine energy                 0.882448
Mixed Hydro Plants            0.603865
Natural gas                   0.990338
Nuclear                       0.502415
Offshore wind energy          0.758454
Oil                           0.962963
Onshore wind energy           0.083736
Other non-renewable energy    0.273752
Pumped storage                0.484702
Renewable hydropower          0.074074
Renewable municipal waste     0.386473
Solar photovoltaic            0.199678
Solar thermal energy          0.916264
Solid biofuels                0.140097
dtype: float64

#### Doing off-grid capacity

In [10]:
amount_str = "Off-grid Capacity (MW)"
offgrid = pd.read_excel("InstalledElectricytCapacity_Offgrid.xlsx",skiprows=1)

#remove the last few rows
offgrid.drop(offgrid.tail(26).index, inplace=True)

#get rid of the column that just says "off-grid"
offgrid.drop("Unnamed: 2",inplace=True,axis=1)

#rename the columns
offgrid = offgrid.rename(columns = {"Unnamed: 0": "Country",
                                  "Unnamed: 1": "Technology",
                                  "Unnamed: 3": "Year",
                                  "Unnamed: 4": amount_str})

#fill in the blanks
new_offgrid = fill_blanks(offgrid)

#change the shape
new_new_offgrid = new_offgrid.pivot_table(index=["Country","Year"], columns='Technology', values = amount_str, aggfunc='sum')
# new_new_offgrid.to_csv("offgrid_cap.csv")

In [11]:
new_new_ongrid["Solar photovoltaic"] = new_new_offgrid["Solar photovoltaic"] + new_new_ongrid["Solar photovoltaic"]

In [12]:
new_new_ongrid.to_csv("grid_cap.csv")

In [149]:
(new_new_offgrid == 0).astype(int).sum(axis=0)/np.shape(new_new_offgrid)[0]

Technology
Biogas                        1.000000
Coal and peat                 1.000000
Fossil fuels n.e.s.           1.000000
Geothermal energy             1.000000
Liquid biofuels               1.000000
Marine energy                 1.000000
Mixed Hydro Plants            1.000000
Natural gas                   1.000000
Nuclear                       1.000000
Offshore wind energy          1.000000
Oil                           1.000000
Onshore wind energy           1.000000
Other non-renewable energy    1.000000
Pumped storage                1.000000
Renewable hydropower          1.000000
Renewable municipal waste     1.000000
Solar photovoltaic            0.864734
Solar thermal energy          1.000000
Solid biofuels                1.000000
dtype: float64

#### Public Investments

In [ ]:
amount_str = "Investment amount"
offgrid = pd.read_excel("PublicInvestments.xlsx",skiprows=1)

#remove the last few rows
offgrid.drop(offgrid.tail(24).index, inplace=True)

#rename the columns
df = offgrid.rename(columns = {"Unnamed: 0": "Country",
                                  "Unnamed: 1": "Technology",
                                  "Unnamed: 2": "Year",
                                  "Unnamed: 3": amount_str})


"""This takes a dataframe (assuming there's a column called "Country") and fills in all the blank country and technologyspots"""
all_countries = list(df["Country"].dropna())

for i in range(len(all_countries)-1):
    start_index = df[df["Country"] == all_countries[i]].index[0]
    end_index = df[df["Country"] == all_countries[i+1]].index[0]
    df.loc[start_index:end_index-1, "Country"] = all_countries[i]
    
#fill the last subset
df["Country"] = df["Country"].fillna(all_countries[-1])

#get just the EU countries
df = df[df["Country"].isin(EU_countries)]

#time to fill in the technologies
n = len(EU_countries)
technologies = list(df[df["Country"] == EU_countries[0]]["Technology"].dropna())
new_df = df.copy(deep=True)


for j in range(n-1): #for each country
    for i in range(len(technologies)-1): #for each technology
        #find the location where the 
        
        start_index = df[df["Technology"] == technologies[i]].index[j]
        end_index = df[df["Technology"] == technologies[i+1]].index[j]

        new_df.loc[start_index:end_index-1, "Technology"] = technologies[i]
    
new_df["Technology"].fillna(technologies[-1],inplace=True)


# #change the shape
# new_new_offgrid = new_offgrid.pivot_table(index=["Country","Year"], columns='Technology', values = amount_str, aggfunc='sum')
# # new_new_offgrid.to_csv("offgrid_cap.csv")

In [186]:
#change the shape
new_new_df = new_df.pivot_table(index=["Country","Year"], columns='Technology', values=amount_str,aggfunc='sum')
new_new_df.to_csv("pubinv.csv")

In [185]:
new_new_df

Technology      Biogas  Coal and peat  Concentrated solar power  \
Country Year                                                      
Austria 2000.0    0.00            0.0                       0.0   
        2001.0    0.00            0.0                       0.0   
        2002.0    0.00            0.0                       0.0   
        2003.0    0.00            0.0                       0.0   
        2004.0    0.00            0.0                       0.0   
...                ...            ...                       ...   
Sweden  2017.0    0.00            0.0                       0.0   
        2018.0    0.00            0.0                       0.0   
        2019.0   96.39            0.0                       0.0   
        2020.0    0.00            0.0                       0.0   
        2021.0    0.00            0.0                       0.0   

Technology      Fossil fuels n.e.s.  Geothermal energy  Liquid biofuels  \
Country Year                                                              
Austria 2000.0                  0.0                0.0              0.0   
        2001.0                  0.0                0.0              0.0   
        2002.0                  0.0                0.0              0.0   
        2003.0                  0.0                0.0              0.0   
        2004.0                  0.0                0.0              0.0   
...                             ...                ...              ...   
Sweden  2017.0                  0.0                0.0              0.0   
        2018.0                  0.0                0.0              0.0   
        2019.0                  0.0                0.0              0.0   
        2020.0                  0.0                0.0              0.0   
        2021.0                  0.0                0.0              0.0   

Technology      Marine energy  Multiple renewables*  Natural gas  \
Country Year                                                       
Austria 2000.0            0.0                   0.0          0.0   
        2001.0            0.0                   0.0          0.0   
        2002.0            0.0                   0.0          0.0   
        2003.0            0.0                   0.0          0.0   
        2004.0            0.0                   0.0          0.0   
...                       ...                   ...          ...   
Sweden  2017.0            0.0                   0.0          0.0   
        2018.0            0.0                   0.0          0.0   
        2019.0            0.0                   0.0          0.0   
        2020.0            0.0                   0.0          0.0   
        2021.0            0.0                   0.0          0.0   

Technology      Non-renewable municipal waste  ...  \
Country Year                                   ...   
Austria 2000.0                            0.0  ...   
        2001.0                            0.0  ...   
        2002.0                            0.0  ...   
        2003.0                            0.0  ...   
        2004.0                            0.0  ...   
...                                       ...  ...   
Sweden  2017.0                            0.0  ...   
        2018.0                            0.0  ...   
        2019.0                            0.0  ...   
        2020.0                            0.0  ...   
        2021.0                            0.0  ...   

Technology      Off-grid Solar photovoltaic  Offshore wind energy  Oil  \
Country Year                                                             
Austria 2000.0                          0.0                   0.0  0.0   
        2001.0                          0.0                   0.0  0.0   
        2002.0                          0.0                   0.0  0.0   
        2003.0                          0.0                   0.0  0.0   
        2004.0                          0.0                   0.0  0.0   
...                                     ...   